# Influx DB

In [8]:
!pip install influxdb-client

## Write some data

#### Connect to Influx server

In [9]:
INFLUXDB_TOKEN="DuT0V2YuYT5z3FMpu6iOcJtQE9RoHYRMlR8BqV4mVVrUJBFoYIJOY7zGE4xJpZ8nglSnxwDNt531685zbwYYuQ==" # My token is different from your, retrieve from influx python interface
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = INFLUXDB_TOKEN
org = "org"
url = "http://localhost:8086"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)


#### Write data to the default Home Assistant bucket

In [10]:
bucket="test"

write_api = client.write_api(write_options=SYNCHRONOUS)
   
for value in range(5):
  point = (
    Point("measurement1")
    .tag("tagname1", "tagvalue1")
    .field("field1", value)
  )
  write_api.write(bucket=bucket, org="org", record=point)
  time.sleep(1) # separate points by 1 second

#### Execute a query on added data

In [11]:
query_api = client.query_api()

query = """
    from(bucket: "test")
    |> range(start: -10m)
    |> filter(fn: (r) => r._measurement == "measurement1")
"""
tables = query_api.query(query, org="org")

for table in tables:
  for record in table.records:
    print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2022, 11, 17, 8, 56, 29, 129567, tzinfo=tzutc()), '_stop': datetime.datetime(2022, 11, 17, 9, 6, 29, 129567, tzinfo=tzutc()), '_time': datetime.datetime(2022, 11, 17, 9, 6, 14, 424617, tzinfo=tzutc()), '_value': 0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2022, 11, 17, 8, 56, 29, 129567, tzinfo=tzutc()), '_stop': datetime.datetime(2022, 11, 17, 9, 6, 29, 129567, tzinfo=tzutc()), '_time': datetime.datetime(2022, 11, 17, 9, 6, 15, 575549, tzinfo=tzutc()), '_value': 1, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2022, 11, 17, 8, 56, 29, 129567, tzinfo=tzutc()), '_stop': datetime.datetime(2022, 11, 17, 9, 6, 29, 129567, tzinfo=tzutc()), '_time': datetime.datetime(2022, 11

#### Run a query on aggregate data

In [12]:
query_api = client.query_api()

query = """
    from(bucket: "test")
    |> range(start: -10m)
    |> filter(fn: (r) => r._measurement == "measurement1")
    |> mean()
"""
tables = query_api.query(query, org="org")

for table in tables:
  for record in table.records:
    print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2022, 11, 17, 8, 56, 29, 207333, tzinfo=tzutc()), '_stop': datetime.datetime(2022, 11, 17, 9, 6, 29, 207333, tzinfo=tzutc()), '_value': 2.0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
